In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import skipgrams
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Lambda

plt.rcParams['figure.figsize'] = (20, 8)

In [2]:
# Dados de exemplo
corpus = [
    'Quais foram os 5 maiores fornecedores de janeiro a março de 2024?', 
    'Qual o total e percentual de pagamentos manuais ocorridos em janeiro de 2024?', 
    'Quanto tenho a pagar para o fornecedor Google?', 
    'Quanto tenho a pagar para os fornecedores X, Y, Z?', 
    'Quanto tenho a pagar para fornecedores do grupo 4000 (Jurídico)?', 
    'Quanto paguei ontem para a empresa 1000?', 
    'Qual o total que tenho a pagar para a empresa 1100 hoje?', 
    'Pagamentos manuais ou pagamento manual (Z, E, B).', 
    'Pagamentos (total, listagem de, etc.) efetuados após o vencimento, considerando-se calendário FEBRABAN, por empresa e intervalo de tempo.', 
    'Percentual de pagamentos efetuados após o vencimento, considerando o total de vencimentos.', 
    'Documentos vencidos, por empresa, por intervalo de tempo, por fornecedor, etc., considerando-se o calendário FEBRABAN.', 
    'Relação de documentos (ou total) a vencer (considerando calendário FEBRABAN) em dado período de tempo.', 
    'Qual o total de pagamentos efetuados após o vencimento pela empresa X no período de janeiro a março de 2024?', 
    'Listagem de pagamentos efetuados após o vencimento para o fornecedor Y considerando calendário FEBRABAN.', 
    'Percentual de documentos vencidos para o grupo 4000 no primeiro trimestre de 2024.', 
    'Relação de documentos a vencer para o fornecedor Google no mês de fevereiro de 2024.', 
    'Total de pagamentos vencidos por empresa 1000 no ano de 2023.', 
    'Percentual de pagamentos manuais feitos para os fornecedores X, Y e Z em janeiro de 2024.', 
    'Quais são os documentos a vencer para o grupo Jurídico na próxima semana?', 
    'Qual o total de pagamentos efetuados em janeiro de 2024?', 
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?', 
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?', 
    'Qual foi o valor total pago para o fornecedor Microsoft no mês de março de 2024?', 
    'Quanto tenho a pagar para fornecedores do grupo 3000 (TI)?', 
    'Quanto paguei anteontem para a empresa 2000?', 
    'Qual o total que tenho a pagar para a empresa 2100 amanhã?', 
    'Pagamentos manuais efetuados para (A, B, C) no primeiro trimestre de 2024.', 
    'Quais foram os 10 maiores fornecedores de fevereiro a abril de 2024?', 
    'Quanto foi pago em junho de 2024 para fornecedores do grupo 5000 (Financeiro)?',
    'Qual o valor total de pagamentos vencidos para a empresa 3100 em maio de 2024?.', 
    'Qual é a lista de fornecedores do grupo 6000 (Marketing) no mês de julho de 2024?', 
    'Quantos documentos venceram em agosto de 2024 para o fornecedor Amazon?', 
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.', 
    'Qual é a relação de documentos a vencer para a empresa 9000 no próximo semestre?', 
    'Quantos pagamentos manuais foram feitos para o fornecedor Oracle no primeiro trimestre de 2023?', 
    'Qual o total de documentos a vencer no mês de fevereiro de 2023 para o grupo 1200 (Comercial)?', 
    'Qual o percentual de pagamentos efetuados após o vencimento para a empresa 8000 no quarto trimestre de 2024?', 
    'Quanto foi pago para fornecedores do grupo 1100 (Administrativo) no último semestre?', 
    'Relação de documentos vencidos para a empresa 7000 considerando o calendário FEBRABAN.', 
    'Quanto tenho a pagar para fornecedores do grupo 1000 (TI) no próximo trimestre?', 
    'Quais são os fornecedores do grupo 9000 (Logística) que têm documentos a vencer no próximo mês?', 
    'Qual o valor total de documentos vencidos para o fornecedor IBM no ano de 2022?', 
    'Quantos pagamentos manuais foram efetuados para a empresa 6000 no mês passado?', 
    'Qual é o percentual de pagamentos efetuados manualmente para fornecedores do grupo 8000 (Operacional) em dezembro de 2024?',
    'Quanto foi pago para o fornecedor Facebook no ano de 2023?', 
    'Qual o total de documentos a vencer em novembro de 2024 para a empresa 5000?', 
    'Quais são os fornecedores que mais receberam pagamentos manuais em outubro de 2024?', 
    'Percentual de documentos vencidos para o grupo 7000 (RH) no segundo trimestre de 2024.', 
    'Qual o total de pagamentos efetuados após o vencimento pela empresa 4000 no período de abril a junho de 2024?', 
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.'
]

# Tokenização
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word2id = tokenizer.word_index
id2word = {v: k for k, v in word2id.items()}
vocab_size = len(word2id) + 1  # +1 para a palavra desconhecida

print('palavra para id: ',word2id)
print('id para palavra: ',id2word)

palavra para id:  {'de': 1, 'o': 2, 'a': 3, 'para': 4, '2024': 5, 'pagamentos': 6, 'no': 7, 'qual': 8, 'total': 9, 'empresa': 10, 'documentos': 11, 'em': 12, 'fornecedores': 13, 'grupo': 14, 'quanto': 15, 'efetuados': 16, 'percentual': 17, 'manuais': 18, 'fornecedor': 19, 'vencer': 20, 'do': 21, 'considerando': 22, 'os': 23, 'tenho': 24, 'pagar': 25, 'calendário': 26, 'febraban': 27, 'vencidos': 28, 'após': 29, 'vencimento': 30, 'relação': 31, 'trimestre': 32, 'mês': 33, 'quais': 34, 'janeiro': 35, 'por': 36, 'fevereiro': 37, 'foram': 38, 'e': 39, 'que': 40, '2023': 41, 'feitos': 42, 'foi': 43, 'pago': 44, 'março': 45, 'x': 46, 'y': 47, 'z': 48, '4000': 49, '1000': 50, 'tempo': 51, 'período': 52, 'primeiro': 53, 'ano': 54, 'são': 55, 'valor': 56, 'é': 57, 'quantos': 58, 'próximo': 59, 'maiores': 60, 'google': 61, 'jurídico': 62, 'paguei': 63, '1100': 64, 'ou': 65, 'b': 66, 'listagem': 67, 'etc': 68, 'se': 69, 'intervalo': 70, 'pela': 71, 'ti': 72, 'abril': 73, 'junho': 74, '5000': 75, 

In [3]:
def generate_cbow_data(sentences, window_size, vocab_size):
    X = []
    y = []
    for sentence in sentences:
        encoded = tokenizer.texts_to_sequences([sentence])[0]
        for i, word_id in enumerate(encoded):
            context_ids = []
            for j in range(max(0, i - window_size), min(len(encoded), i + window_size + 1)):
                if j != i:
                    context_ids.append(encoded[j])
            if context_ids:
                X.append(context_ids)
                y.append(word_id)
    
    # Pad sequences to ensure fixed length
    X_padded = pad_sequences(X, maxlen=window_size*2)
    
    return np.array(X_padded), np.array(y)

# Exemplo de uso
window_size = 2
X_train, y_train = generate_cbow_data(corpus, window_size, vocab_size)

In [4]:
# Construção do modelo CBOW
embedding_dim = 128
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=window_size*2))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 128)            16768     
                                                                 
 lambda (Lambda)             (None, 128)               0         
                                                                 
 dense (Dense)               (None, 131)               16899     
                                                                 
Total params: 33667 (131.51 KB)
Trainable params: 33667 (131.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# Treinamento do modelo
model.fit(X_train, y_train, epochs=1000, verbose=True)

Epoch 1/1000
21/21 [==============================] - 0s 1ms/step - loss: 4.8562
Epoch 2/1000
21/21 [==============================] - 0s 857us/step - loss: 4.7962
Epoch 3/1000
21/21 [==============================] - 0s 843us/step - loss: 4.7208
Epoch 4/1000
21/21 [==============================] - 0s 821us/step - loss: 4.6148
Epoch 5/1000
21/21 [==============================] - 0s 823us/step - loss: 4.4690
Epoch 6/1000
21/21 [==============================] - 0s 831us/step - loss: 4.2873
Epoch 7/1000
21/21 [==============================] - 0s 826us/step - loss: 4.0932
Epoch 8/1000
21/21 [==============================] - 0s 799us/step - loss: 3.9101
Epoch 9/1000
21/21 [==============================] - 0s 771us/step - loss: 3.7577
Epoch 10/1000
21/21 [==============================] - 0s 776us/step - loss: 3.6226
Epoch 11/1000
21/21 [==============================] - 0s 780us/step - loss: 3.4924
Epoch 12/1000
21/21 [==============================] - 0s 786us/step - loss: 3.3631
Epo

In [6]:
# Obter embeddings
embeddings = model.layers[0].get_weights()[0]
print(f"Embedding da palavra 'fornecedores': ")
print(embeddings[word2id['fornecedores']])

Embedding da palavra 'fornecedores': 
[ 0.35479787 -0.00908266  0.14055382  0.7322974  -0.29137415 -0.37067497
  0.5031903  -0.8558192  -0.00444756  0.39630052 -0.5389731  -0.20167522
  0.5963156   0.7665127  -0.16082732  0.09056383 -0.29381987  0.92982256
  0.37995788 -1.2983826   0.5445948   1.3069423   0.79395044  1.2125567
  0.8897174  -0.34356147  0.03838937 -1.1497823   1.5280813   0.496733
  0.8004299  -1.2564963  -0.62520105  0.22520658  1.1310645  -1.3207352
 -0.22767651 -0.47347337 -0.33841234 -0.99091506  0.2451703  -1.3722425
  0.7481323  -0.7763142  -0.9320571   0.544843    0.47079855 -0.48026344
  0.77175176  0.09011755 -1.4994705  -0.5282702  -0.32311895 -0.20198242
 -0.3039403  -1.5704854  -0.10521674 -0.76551926 -0.1124892   0.25394228
 -0.08841126  0.51247364  0.33605847  0.5055264  -1.2511698  -0.7223259
 -0.15563072 -0.05107114  0.09157649 -0.21334274 -0.11938111  0.3567394
  0.24139278  0.54941744  0.99503875  0.25378805  0.38482156 -1.4101185
  0.9016639   0.94059

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Exemplo de corpus anotado para NER (simplificado)
texts = [
    'Quais foram os 5 maiores fornecedores de janeiro a março de 2024?', 
    'Qual o total e percentual de pagamentos manuais ocorridos em janeiro de 2024?', 
    'Quanto tenho a pagar para o fornecedor Google?', 
    'Quanto tenho a pagar para os fornecedores X, Y, Z?', 
    'Quanto tenho a pagar para fornecedores do grupo 4000 (Jurídico)?', 
    'Quanto paguei ontem para a empresa 1000?', 
    'Qual o total que tenho a pagar para a empresa 1100 hoje?', 
    'Pagamentos manuais ou pagamento manual (Z, E, B).', 
    'Pagamentos (total, listagem de, etc.) efetuados após o vencimento, considerando-se calendário FEBRABAN, por empresa e intervalo de tempo.', 
    'Percentual de pagamentos efetuados após o vencimento, considerando o total de vencimentos.', 
    'Documentos vencidos, por empresa, por intervalo de tempo, por fornecedor, etc., considerando-se o calendário FEBRABAN.', 
    'Relação de documentos (ou total) a vencer (considerando calendário FEBRABAN) em dado período de tempo.', 
    'Qual o total de pagamentos efetuados após o vencimento pela empresa X no período de janeiro a março de 2024?', 
    'Listagem de pagamentos efetuados após o vencimento para o fornecedor Y considerando calendário FEBRABAN.', 
    'Percentual de documentos vencidos para o grupo 4000 no primeiro trimestre de 2024.', 
    'Relação de documentos a vencer para o fornecedor Google no mês de fevereiro de 2024.', 
    'Total de pagamentos vencidos por empresa 1000 no ano de 2023.', 
    'Percentual de pagamentos manuais feitos para os fornecedores X, Y e Z em janeiro de 2024.', 
    'Quais são os documentos a vencer para o grupo Jurídico na próxima semana?', 
    'Qual o total de pagamentos efetuados em janeiro de 2024?', 
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?', 
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?', 
    'Qual foi o valor total pago para o fornecedor Microsoft no mês de março de 2024?', 
    'Quanto tenho a pagar para fornecedores do grupo 3000 (TI)?', 
    'Quanto paguei anteontem para a empresa 2000?', 
    'Qual o total que tenho a pagar para a empresa 2100 amanhã?', 
    'Pagamentos manuais efetuados para (A, B, C) no primeiro trimestre de 2024.', 
    'Quais foram os 10 maiores fornecedores de fevereiro a abril de 2024?', 
    'Quanto foi pago em junho de 2024 para fornecedores do grupo 5000 (Financeiro)?',
    'Qual o valor total de pagamentos vencidos para a empresa 3100 em maio de 2024?.', 
    'Qual é a lista de fornecedores do grupo 6000 (Marketing) no mês de julho de 2024?', 
    'Quantos documentos venceram em agosto de 2024 para o fornecedor Amazon?', 
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.', 
    'Qual é a relação de documentos a vencer para a empresa 9000 no próximo semestre?', 
    'Quantos pagamentos manuais foram feitos para o fornecedor Oracle no primeiro trimestre de 2023?', 
    'Qual o total de documentos a vencer no mês de fevereiro de 2023 para o grupo 1200 (Comercial)?', 
    'Qual o percentual de pagamentos efetuados após o vencimento para a empresa 8000 no quarto trimestre de 2024?', 
    'Quanto foi pago para fornecedores do grupo 1100 (Administrativo) no último semestre?', 
    'Relação de documentos vencidos para a empresa 7000 considerando o calendário FEBRABAN.', 
    'Quanto tenho a pagar para fornecedores do grupo 1000 (TI) no próximo trimestre?', 
    'Quais são os fornecedores do grupo 9000 (Logística) que têm documentos a vencer no próximo mês?', 
    'Qual o valor total de documentos vencidos para o fornecedor IBM no ano de 2022?', 
    'Quantos pagamentos manuais foram efetuados para a empresa 6000 no mês passado?', 
    'Qual é o percentual de pagamentos efetuados manualmente para fornecedores do grupo 8000 (Operacional) em dezembro de 2024?',
    'Quanto foi pago para o fornecedor Facebook no ano de 2023?', 
    'Qual o total de documentos a vencer em novembro de 2024 para a empresa 5000?', 
    'Quais são os fornecedores que mais receberam pagamentos manuais em outubro de 2024?', 
    'Percentual de documentos vencidos para o grupo 7000 (RH) no segundo trimestre de 2024.', 
    'Qual o total de pagamentos efetuados após o vencimento pela empresa 4000 no período de abril a junho de 2024?', 
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.'
]
ner_tags = [
    'O O O MIN_MAX MIN_MAX TABELA O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O O FORNECEDOR?', 
    'O O O O O O TABELA FORNECEDOR, FORNECEDOR, FORNECEDOR?', 
    'O O O O O TABELA O O ID O', 
    'O O O O O O ID?', 
    'O O O O O O O O O O ID INTERVALO?', 
    'TABELA CONDICAO CONDICAO O O (FORNECEDOR, FORNECEDOR, FORNECEDOR).', 
    'TABELA O O O O O CONDICAO CONDICAO, O O O O O O O O O O', 
    'O O TABELA O CONDICAO CONDICAO, O O O O O O', 
    'TABELA CONDICAO CONDICAO, O O O O O O O O O O O O', 
    'O O TABELA O O CONDICAO CONDICAO O O O O O O O O', 
    'O O O O TABELA O CONDICAO CONDICAO O O FORNECEDOR O O O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA O CONDICAO CONDICAO O O O FORNECEDOR O O O O', 
    'O O TABELA CONDICAO O O O ID O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO CONDICAO O O O FORNECEDOR O O O INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O ID INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O O O TABELA FORNECEDOR, FORNECEDOR O FORNECEDOR O INTERVALO INTERVALO INTERVALO?', 
    'O O O TABELA CONDICAO CONDICAO O O O O O INTERVALO INTERVALO?', 
    'O O O O TABELA O O INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O O O O FORNECEDOR O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O O TABELA O O ID O', 
    'O O INTERVALO O O O ID?', 
    'O O O O O O O O O O ID INTERVALO?', 
    'TABELA CONDICAO CONDICAO O (FORNECEDOR, FORNECEDOR, FORNECEDOR) O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O MIN_MAX MIN_MAX TABELA O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O INTERVALO INTERVALO INTERVALO O TABELA O O ID O', 
    'O O O O O TABELA CONDICAO O O O ID O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O TABELA O O ID O O O O INTERVALO INTERVALO INTERVALO?', 
    'O TABELA CONDICAO O INTERVALO INTERVALO INTERVALO O O O FORNECEDOR?', 
    'O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO O O O O', 
    'O O O O O TABELA CONDICAO CONDICAO O O O ID O INTERVALO ?', 
    'O TABELA CONDICAO O O O O O FORNECEDOR O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O O O INTERVALO INTERVALO INTERVALO O O O ID O', 
    'O O O O TABELA O CONDICAO CONDICAO CONDICAO O O O ID O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA O O ID O O INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O O O ID O O O O', 
    'O O O O O TABELA O O ID O O INTERVALO INTERVALO?', 
    'O O O TABELA O O ID O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO?', 
    'O O O O O TABELA CONDICAO CONDICAO O O FORNECEDOR O INTERVALO INTERVALO INTERVALO?', 
    'O TABELA CONDICAO O O O O O ID O INTERVALO INTERVALO?', 
    'O O O O O TABELA O CONDICAO O TABELA O O ID O O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O FORNECEDOR O INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO INTERVALO O O ID?', 
    'O O O TABELA O O O TABELA CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O O O ID O O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA O CONDICAO CONDICAO CONDICAO O O ID O O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO O O O O'
]

# Tokenização do texto e tags
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, padding='post')

# Tokenização das tags NER
tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(ner_tags)
y = tag_tokenizer.texts_to_sequences(ner_tags)
y = pad_sequences(y, padding='post')
word2id_NER = tag_tokenizer.word_index
id2word_NER = {v: k for k, v in word2id_NER.items()}

In [8]:
# Verificando as formas de X e y
print("Forma de X:", X.shape)
print("Forma de y:", y.shape)

Forma de X: (50, 20)
Forma de y: (50, 20)


In [9]:
# Definir o modelo NER
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
print(vocab_size)

131


In [10]:
input_layer = Input(shape=(X.shape[1],))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
output_layer = Dense(len(tag_tokenizer.word_index) + 1, activation='softmax')(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 20, 100)           13100     
                                                                 
 bidirectional (Bidirection  (None, 20, 128)           84480     
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 20, 9)             1161      
                                                                 
Total params: 98741 (385.71 KB)
Trainable params: 98741 (385.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Treinar o modelo
model.fit(X, y, epochs=10000, batch_size=64)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 2.1962 - accuracy: 0.0460
Epoch 2/10000
1/1 [==============================] - 0s 17ms/step - loss: 2.1760 - accuracy: 0.5670
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 2.1554 - accuracy: 0.6670
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 2.1342 - accuracy: 0.6970
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 2.1118 - accuracy: 0.7100
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 2.0879 - accuracy: 0.7140
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 2.0620 - accuracy: 0.7120
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 2.0335 - accuracy: 0.7130
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 2.0018 - accuracy: 0.7130
Epoch 10/10000
1/1 [==============================] - 0s 12ms/step - loss: 1.9661 - accuracy: 0.7120
E

In [ ]:
# Função para prever NER tags
def predict_ner(text):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=X.shape[1], padding='post')
    pred = model.predict(padded_seq)[0]
    pred_tags = np.argmax(pred, axis=-1)
    return [tag_tokenizer.index_word[tag] for tag in pred_tags if tag != 0]

def ner_tags(text):
    tags = predict_ner(text)
    print(text)
    for sentense_x in tokenizer.texts_to_sequences([text]):   
            word = [id2word[id] for id in sentense_x if id in id2word]
            for w in range(len(word)):
                if not tags[w] == 'o':
                    print(f' - {word[w]}: {tags[w]}')
test_texts = [
    'Quais foram os fornecedores que mais venderam no último trimestre?',
    'Quantos pagamentos foram feitos manualmente em março deste ano?',
    'Qual a lista de fornecedores do grupo 2000?',
    'Como está o percentual de documentos vencidos este mês?',
    'Quem são os fornecedores com maior número de pagamentos em atraso?',
    'Quantos pagamentos foram efetuados após o vencimento no primeiro semestre de 2024?',
    'Qual é a relação de documentos a vencer em julho de 2024?'
]
# Exemplo de previsão
for text in test_texts:
    ner_tags(text)